Imports

Wait why is there a car n02979186_603 in n02979186.

In [1]:
import torch
import torch.nn as nn

from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
from fastai.vision.all import *
set_seed(42,reproducible=True)
source = untar_data(URLs.IMAGENETTE)

In [4]:
# train_dataset_path = './imagenette2/train'
# val_dataset_path = './imagenette2/val'
train_dataset_path = source/'train'
val_dataset_path = source/'val'

img_height = 224
img_width = 224
input_size = img_height * img_width 

# # 256x256
# mean = [0.4625, 0.4580, 0.4295]
# std = [0.2362, 0.2298, 0.2383]

# calculated in mean_and_std_calc 224x224
mean = [0.4625, 0.4580, 0.4295]
std = [0.2351, 0.2287, 0.2372]

In [5]:
train_transforms = transforms.Compose([
  # transforms.Resize((img_height, img_width)),
  transforms.RandomHorizontalFlip(),
  transforms.RandomRotation(10),
  transforms.RandomResizedCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])
# consider including scale and aspect ratio, randomCrop,  randomPerspective, randomErase

val_transforms = transforms.Compose([
  transforms.Resize((img_height, img_width)),
  transforms.ToTensor(),
  transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

Hyperparameters

In [6]:
batch_size=64
learning_rate=0.001 #  check out learning  rate decay

Load in data

In [7]:
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
val_dataset = torchvision.datasets.ImageFolder(root=val_dataset_path, transform = val_transforms)

In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size, shuffle=False)

In [9]:
class Block(nn.Module):
  def __init__(self, input_nodes, intermediate_nodes, downsample):
    super(Block, self).__init__()
    self.conv1 = nn.Conv2d(input_nodes, intermediate_nodes, kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(intermediate_nodes)
    self.conv2 = nn.Conv2d(intermediate_nodes, intermediate_nodes, kernel_size=3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(intermediate_nodes)
    self.relu = nn.ReLU()
    self.downsample = downsample

  def forward(self, input):
    original = input.clone()
    x = self.conv1(input)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    if self.downsample is not None:
      original = self.downsample(original)
    x += original
    x = self.relu(x)
    return x

class Model(nn.Module):
  def __init__(self, num_features=3, num_classes=10, num_hidden1=64, num_hidden2=128, num_hidden3=256, num_hidden4=512, dropout_prob=0.2):
    super(Model, self).__init__()
    self.num_hidden1 = num_hidden1
    self.initial_split = nn.Conv2d(3, 64, 7, stride=2, padding=1)
    self.maxPool1 = nn.MaxPool2d(3,stride=2)

    self.softmax = nn.Softmax(dim=1)

  def forward(self, input):
    x = self.initial_split(input)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fcnn(x)
    x = self.softmax(x)
    return x

In [10]:
class Block2(nn.Module):
  def __init__(self, in_channels, mid_channels):
    super(Block2, self).__init__()
    # split into 2 branches
    self.conv11 = nn.Conv2d(in_channels,mid_channels*4, 1, stride=2, padding=0, bias=False) # b1: stride downsample
    self.bn11 = nn.BatchNorm2d(mid_channels*4)
    self.conv12 = nn.Conv2d(in_channels, mid_channels, 3, stride=1, padding=1, bias=False) # b2: normal convolution
    self.bn12 = nn.BatchNorm2d(mid_channels)

    self.conv21 = nn.Conv2d(mid_channels*4, mid_channels*4, 3, stride=1, padding=1, bias=False) # b1: perform convolution on downsampled
    self.bn21 = nn.BatchNorm2d(mid_channels*4)
    self.conv22 = nn.Conv2d(mid_channels, mid_channels, 3, stride=1, padding=1, bias=False) # b2: perform second normal convolution
    self.bn22 = nn.BatchNorm2d(mid_channels)

    self.conv32 = nn.Conv2d(mid_channels, mid_channels*4, 3, stride=1, padding=1, bias=False) # b2: increase channels to match b1
    self.bn32 = nn.BatchNorm2d(mid_channels*4)
    self.maxPool42 = nn.MaxPool2d(2)

    self.relu = nn.ReLU()
    pass
  
  def forward(self, input):
    """
    layer 1 64 x 56x56
    
    traverse it 

    create a 26*26 final image
    """
    x = self.conv11(input)
    x = self.bn11(x)
    x = self.relu(x)
    x = self.conv21(x)
    x = self.bn21(x)
    x = self.relu(x)

    y = self.conv12(input)
    y = self.bn12(y)
    y = self.relu(y)
    y = self.conv22(y)
    y = self.bn22(y)
    y = self.conv32(y)
    y = self.bn32(y)
    y = self.relu(y)
    y = self.maxPool42(y)
    combined = x+y
    combined = self.relu(combined)
    return combined

class Model2(nn.Module):
  def __init__(self, in_channels=3, num_classes=10):
    super(Model2, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.layer1 = Block2(64, 32)
    self.layer2 = Block2(128,64)
    self.layer3 = Block2(256,128)
    self.fcnn = nn.Linear(7*7, num_classes)
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.flatten = nn.Flatten()
    self.fc = nn.Linear(512, num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    # x = self.layer4(x)
    x = self.avgpool(x)
    x = self.flatten(x)
    
    x = self.fc(x)
    # softmax this
    return x

In [11]:
#@title ResNet: Not own code (20 epochs, % training, 17% testing)

class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False
        ) # [_, int_channels, x,y]
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        ) # [_, int_channels, x,y]
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        ) # [_, int_channels*4, x, y]
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x


class ResNet(nn.Module):
    def __init__(self, block, layers, image_channels, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Essentially the entire ResNet architecture are in these 4 lines below
        self.layer1 = self._make_layer(
            block, layers[0], intermediate_channels=64, stride=1
        )
        self.layer2 = self._make_layer(
            block, layers[1], intermediate_channels=128, stride=2
        )
        self.layer3 = self._make_layer(
            block, layers[2], intermediate_channels=256, stride=2
        )
        self.layer4 = self._make_layer(
            block, layers[3], intermediate_channels=512, stride=2
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, block, num_residual_blocks, intermediate_channels, stride):
        identity_downsample = None
        layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != intermediate_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    intermediate_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(intermediate_channels * 4),
            )

        layers.append(
            block(self.in_channels, intermediate_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = intermediate_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            layers.append(block(self.in_channels, intermediate_channels))

        return nn.Sequential(*layers)

In [12]:
# model = ResNet(block, [3,4,6,3], image_channels=3, num_classes=10).to(device)
model = Model2().to(device)

Loss Function

In [14]:
criterion = torch.nn.CrossEntropyLoss()
# optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0001)

Training Loop

In [ ]:
num_epochs = 20
train_count = 9469
test_count = 3925

In [ ]:
best_accuracy = 0.
loss_vctr = []
start = time.time()

for epoch in range(num_epochs):
  model.train()
  train_accuracy = 0.
  train_loss = 0.

  for i,(images,labels) in enumerate(train_loader):
    if torch.cuda.is_available():
      images = Variable(images.cuda())
      labels = Variable(labels.cuda())
    optimiser.zero_grad()
    
    outputs = model(images)
    loss=criterion(outputs,labels)
    loss.backward()
    optimiser.step()

    train_loss += loss.item()

#     if i % 10 == 9:    # print every 2000 mini-batches
#       loss_vctr.append(train_loss / 2000)
#       print('[%d, %5d] loss: %.6f' %
#             (epoch + 1, i + 1, train_loss / 2000))
#       train_loss = 0.0
# print("^^^^^^^^^^^^^^^^^")
# print('Finished Optimization.')
# end = time.time() # Time counted in seconds
# print("The total time to train the model on the K80 GPU is : {:.1f} minutes.".format((end - start)/60))
    
    _,prediction = torch.max(outputs.data, 1)

    train_accuracy += int(torch.sum(prediction==labels.data))

  train_accuracy = train_accuracy / train_count
  train_loss = train_loss / train_count

  model.eval()

  test_accuracy = 0.
  for i, (images,labels) in enumerate(test_loader):
    if torch.cuda.is_available():
      images = Variable(images.cuda())
      labels = Variable(labels.cuda())
    outputs=model(images)
    _,prediction=torch.max(outputs.data,1)
    test_accuracy+=int(torch.sum(prediction==labels.data))
    
  test_accuracy=test_accuracy/test_count
  
  
  print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
  
  #Save the best model
  if test_accuracy>best_accuracy:
      torch.save(model.state_dict(),'best_checkpoint.model')
      best_accuracy=test_accuracy

In [ ]:
correct = 0
total = 0
model.eval()
for data in train_loader:
        images, labels = data
        images = Variable(images).cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.cpu().data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
print('Accuracy of the network on the training images: %d %%' % (
    100 * correct / total))

In [ ]:
correct = 0
total = 0

for data in test_loader:
        images, labels = data
        images = Variable(images).cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.cpu().data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))